# Assignment 3a Solution: Basic Gradio RAG Frontend
## Day 6 Session 2 - Building Simple RAG Applications

This notebook contains the complete solution for Assignment 3a.

**Solution demonstrates:**
- Complete basic Gradio interface setup
- Proper RAG backend integration
- Essential Gradio components and patterns
- Button-function connections


## 📚 Part 1: Setup and Imports

Import all necessary libraries for building your Gradio RAG application.


In [1]:
# Import required libraries
import gradio as gr
import os
from pathlib import Path

# LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

print("✅ All libraries imported successfully!")


/Users/ishandutta/miniconda3/envs/accelerator/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully!


## 🤖 Part 2: RAG Backend Class

The complete RAG backend implementation (provided to students).


In [2]:
class SimpleRAGBackend:
    """Simple RAG backend for Gradio frontend."""
    
    def __init__(self):
        self.index = None
        self.setup_settings()
    
    def setup_settings(self):
        """Configure LlamaIndex settings."""
        # Set up the LLM using OpenRouter
        api_key = os.getenv("OPENROUTER_API_KEY")
        if api_key:
            Settings.llm = OpenRouter(
                api_key=api_key,
                model="gpt-4o",
                temperature=0.1
            )
        
        # Set up the embedding model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            trust_remote_code=True
        )
        
        # Set chunking parameters
        Settings.chunk_size = 1024
        Settings.chunk_overlap = 100
    
    def initialize_database(self, data_folder="../data"):
        """Initialize the vector database with documents."""
        # Check if data folder exists
        if not Path(data_folder).exists():
            return f"❌ Data folder '{data_folder}' not found!"
        
        try:
            # Create vector store
            vector_store = LanceDBVectorStore(
                uri="./basic_rag_vectordb",
                table_name="documents"
            )
            
            # Load documents
            reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
            documents = reader.load_data()
            
            # Create storage context and index
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            self.index = VectorStoreIndex.from_documents(
                documents, 
                storage_context=storage_context,
                show_progress=True
            )
            
            return f"✅ Database initialized successfully with {len(documents)} documents!"
        
        except Exception as e:
            return f"❌ Error initializing database: {str(e)}"
    
    def query(self, question):
        """Query the RAG system and return response."""
        # Check if index exists
        if self.index is None:
            return "❌ Please initialize the database first!"
        
        # Check if question is empty
        if not question or not question.strip():
            return "⚠️ Please enter a question first!"
        
        try:
            # Create query engine and get response
            query_engine = self.index.as_query_engine()
            response = query_engine.query(question)
            return str(response)
        
        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the backend
rag_backend = SimpleRAGBackend()
print("🚀 RAG Backend initialized and ready!")


🚀 RAG Backend initialized and ready!


## 🎨 Part 3: Complete Gradio Interface Solution

This is the complete solution that students should implement.


In [3]:
def create_basic_rag_interface():
    """Create basic RAG interface with essential features."""
    
    def initialize_db():
        """Handle database initialization."""
        return rag_backend.initialize_database()
    
    def handle_query(question):
        """Handle user queries."""
        return rag_backend.query(question)
    
    # Create Gradio interface using gr.Blocks()
    with gr.Blocks(title="Basic RAG Assistant") as interface:
        # Add title and description
        gr.Markdown("# 🤖 Basic RAG Assistant")
        gr.Markdown("Initialize the database and ask questions about your documents!")
        
        # Add initialization section
        init_btn = gr.Button("🔄 Initialize Database", variant="primary")
        status_output = gr.Textbox(label="Status", lines=2, interactive=False)
        
        # Add query section
        query_input = gr.Textbox(
            label="Ask a question", 
            placeholder="What would you like to know?", 
            lines=2
        )
        submit_btn = gr.Button("🚀 Ask Question", variant="primary")
        response_output = gr.Textbox(label="AI Response", lines=10, interactive=False)
        
        # Connect buttons to functions
        init_btn.click(initialize_db, outputs=[status_output])
        submit_btn.click(handle_query, inputs=[query_input], outputs=[response_output])
    
    return interface

# Create the interface
basic_interface = create_basic_rag_interface()
print("✅ Basic RAG interface created successfully!")


✅ Basic RAG interface created successfully!


## 🚀 Part 4: Launch the Application

Launch your Gradio application!


In [ ]:
print("🎉 Launching your Basic RAG Assistant...")
print("🔗 Your application will open in a new browser tab!")
print("")
print("📋 Testing Instructions:")
print("1. Click 'Initialize Database' button first")
print("2. Wait for success message")
print("3. Enter a question in the query box")
print("4. Click 'Ask Question' to get AI response")
print("")
print("💡 Example questions to try:")
print("- What are the main topics in the documents?")
print("- Summarize the key findings")
print("- Explain the methodology used")

# Launch the interface
basic_interface.launch(share=True)


🎉 Launching your Basic RAG Assistant...
🔗 Your application will open in a new browser tab!

📋 Testing Instructions:
1. Click 'Initialize Database' button first
2. Wait for success message
3. Enter a question in the query box
4. Click 'Ask Question' to get AI response

💡 Example questions to try:
- What are the main topics in the documents?
- Summarize the key findings
- Explain the methodology used
* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://1f56b04fd5422ab739.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Table documents doesn't exist yet. Please add some data to create it.
/Users/ishandutta/miniconda3/envs/accelerator/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/ishandutta/miniconda3/envs/accelerator/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/ishandutta/miniconda3/envs/accelerator/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Generating embeddings: 100%|██████████| 55/55 [00:05<00:00, 10.58it/s]
2025-09-21 09:36:31,861 - INFO - Create new table documents adding data.
[2025-09-21T04:06:31Z WARN  lance::dataset::write::insert] No existing dataset at /Users/i

## 🎓 Solution Explanation

### Key Components Used

1. **gr.Blocks()**: Creates a custom layout container
2. **gr.Markdown()**: Adds formatted text titles and descriptions
3. **gr.Button()**: Creates interactive buttons with optional styling
4. **gr.Textbox()**: Creates text input/output areas with customization options
5. **button.click()**: Connects buttons to Python functions

### Important Parameters

- **variant="primary"**: Makes buttons stand out with primary styling
- **lines=N**: Sets the height of textboxes
- **interactive=False**: Makes textboxes read-only for output
- **placeholder**: Adds hint text to input fields
- **label**: Adds descriptive labels to components

### Function Connection Pattern

```python
# Pattern: button.click(function_name, inputs=[input_components], outputs=[output_components])
init_btn.click(initialize_db, outputs=[status_output])
submit_btn.click(handle_query, inputs=[query_input], outputs=[response_output])
```

### Best Practices Demonstrated

1. Clear visual hierarchy with titles and sections
2. Descriptive labels and placeholder text
3. Appropriate component sizes (lines parameter)
4. Read-only outputs to prevent user editing
5. Visual feedback with button variants
6. Logical workflow: initialize → query → response
